In [ ]:
import json

# with open("./wikidata/entityid2label.json", "r", encoding="utf-8") as f:
#     entityid2label = json.load(f)  

# with open("./wikidata/entity_instance_of.json", "r", encoding="utf-8") as f:
#     entity_instance_of = json.load(f)

with open("./wikidata/entity_subclass_of.json", "r", encoding="utf-8") as f:
    entity_subclass_of = json.load(f)

print(f"{len(entity_subclass_of)} entities in entity_subclass_of")

In [ ]:
# Cell: JanusGraph Manager - Add and Delete Data with Streaming

import nest_asyncio
import asyncio
import logging
import random
from itertools import islice
from gremlin_python.driver.client import Client
from gremlin_python.driver.protocol import GremlinServerError
from tqdm.auto import tqdm

# Apply nest_asyncio to allow nested event loops (useful in Jupyter notebooks)
nest_asyncio.apply()

# Configure Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize Gremlin Client
GREMLIN_SERVER_URL = "ws://localhost:8282/gremlin"  # Update if different
client = Client(GREMLIN_SERVER_URL, "g")


def data_generator(entity_dict, slice_size=10000):
    """
    Generator to yield data slices from the entity_instance_of dictionary.

    Args:
        entity_dict (dict): Dictionary mapping child labels to parent labels.
        slice_size (int): Number of entities to process per slice.

    Yields:
        tuple: (set of unique labels, list of (child_label, parent_label) tuples)
    """
    it = iter(entity_dict.items())
    while True:
        slice_data = dict(islice(it, slice_size))
        if not slice_data:
            break
        unique_labels = set()
        relationships = []
        for child_label, parent_labels in slice_data.items():
            unique_labels.add(child_label)
            for parent_label in parent_labels:
                unique_labels.add(parent_label)
                relationships.append((child_label, parent_label))
        yield unique_labels, relationships


async def submit_query(query, retries=10, backoff_in_seconds=1):
    """
    Submit a Gremlin query with retry logic upon encountering lock contention.

    Args:
        query (str): The Gremlin query string.
        retries (int): Number of retries upon failure.
        backoff_in_seconds (int): Initial backoff duration.

    Returns:
        ResultSet: The result of the query.
    """
    attempt = 0
    while attempt <= retries:
        try:
            future = client.submitAsync(query)
            result = await asyncio.wrap_future(future)
            logger.debug(f"Executed query: {query.strip()}")
            return result
        except GremlinServerError as e:
            error_message = e.message.lower()
            if (
                "lock" in error_message
                or "contention" in error_message
                or "expected value mismatch" in error_message
            ):
                if attempt < retries:
                    sleep_time = backoff_in_seconds * (2**attempt) + random.uniform(
                        0, 1
                    )
                    logger.warning(
                        f"Lock contention detected. Retrying in {sleep_time:.2f} seconds... "
                        f"(Attempt {attempt + 1}/{retries})"
                    )
                    await asyncio.sleep(sleep_time)
                    attempt += 1
                else:
                    logger.error(f"Exceeded maximum retries for query: {query.strip()}")
                    raise
            else:
                logger.error(f"Gremlin Server Error: {e.message}")
                raise
        except Exception as e:
            logger.error(f"Unexpected error: {e}")
            raise


async def upsert_vertex(label):
    """
    Insert a vertex with the given label if it doesn't already exist.

    Args:
        label (str): The unique label of the entity.
    """
    query = f"""
        g.V().hasLabel('{label}').fold().
        coalesce(
            unfold(),
            addV('{label}')
        )
    """
    await submit_query(query)


async def upsert_edge(child_label, parent_label):
    """
    Insert an edge 'subclass_of' from child to parent if it doesn't already exist.

    Args:
        child_label (str): Label of the child entity.
        parent_label (str): Label of the parent entity.
    """
    query = f"""
        g.V().hasLabel('{child_label}').as('child').
        V().hasLabel('{parent_label}').as('parent').
        coalesce(
            __.select('child').outE('subclass_of').where(__.inV().as('parent')),
            __.select('child').addE('subclass_of').to('parent')
        )
    """
    await submit_query(query)


async def process_vertices(unique_labels, max_workers=3):
    """
    Process and insert all unique vertices with controlled concurrency.

    Args:
        unique_labels (set): Set of unique entity labels.
        max_workers (int): Number of concurrent tasks for processing.
    """
    semaphore = asyncio.Semaphore(max_workers)

    async def sem_upsert_vertex(label):
        async with semaphore:
            await upsert_vertex(label)

    tasks = [asyncio.create_task(sem_upsert_vertex(label)) for label in unique_labels]

    # Await all tasks and handle exceptions
    for f in tqdm(
        asyncio.as_completed(tasks), total=len(tasks), desc="Upserting Vertices"
    ):
        try:
            await f
        except Exception as e:
            logger.error(f"Error during vertex upsert: {e}")


async def process_edges(relationships, batch_size=5, max_workers=3):
    """
    Process and insert all edges in batches with controlled concurrency.

    Args:
        relationships (list): List of (child, parent) tuples.
        batch_size (int): Number of relationships to process per batch.
        max_workers (int): Number of concurrent tasks for processing.
    """
    total = len(relationships)
    logger.info(f"Total relationships to process: {total}")

    # Split into batches
    for i in range(0, total, batch_size):
        batch = relationships[i : i + batch_size]
        batch_number = i // batch_size + 1
        logger.info(
            f"Processing edge batch {batch_number} with {len(batch)} relationships."
        )
        await _process_edge_batch(batch, max_workers)
        logger.info(f"Edge batch {batch_number} processed.")


async def _process_edge_batch(batch, max_workers):
    """
    Process a single batch of edges using limited concurrency.

    Args:
        batch (list): List of (child, parent) tuples.
        max_workers (int): Number of concurrent tasks for processing.
    """
    semaphore = asyncio.Semaphore(max_workers)

    async def sem_upsert_edge(child, parent):
        async with semaphore:
            await upsert_edge(child, parent)

    tasks = [
        asyncio.create_task(sem_upsert_edge(child, parent)) for child, parent in batch
    ]

    # Await all tasks and handle exceptions
    for f in tqdm(
        asyncio.as_completed(tasks), total=len(tasks), desc="Upserting Edges"
    ):
        try:
            await f
        except Exception as e:
            logger.error(f"Error during edge upsert: {e}")


def get_data_stream(entity_dict, slice_size=10000):
    """
    Create a generator to stream data slices from the entity_instance_of dictionary.

    Args:
        entity_dict (dict): Dictionary mapping child labels to parent labels.
        slice_size (int): Number of entities to process per slice.

    Yields:
        tuple: (set of unique labels, list of (child_label, parent_label) tuples)
    """
    return data_generator(entity_dict, slice_size)


async def delete_all_vertices():
    """
    Delete all vertices (and consequently all edges) from the JanusGraph database.
    """
    delete_query = "g.V().drop().iterate()"
    try:
        await submit_query(delete_query)
        logger.info(
            "All vertices and their associated edges have been successfully deleted."
        )
    except Exception as e:
        logger.error(f"Failed to delete all data: {e}")


async def delete_vertices_in_batches(batch_size=100):
    """
    Delete all vertices in smaller batches to minimize load and lock contention.

    Args:
        batch_size (int): Number of vertices to delete per batch.
    """
    logger.info("Starting deletion of vertices in batches...")
    while True:
        delete_batch_query = f"""
            g.V().limit({batch_size}).drop().iterate()
        """
        try:
            await submit_query(delete_batch_query)
            # Check if any vertices remain
            count_query = "g.V().count()"
            result = await submit_query(count_query)
            remaining = result.one().value
            logger.info(
                f"Deleted a batch of {batch_size} vertices. Remaining vertices: {remaining}"
            )
            if remaining == 0:
                logger.info("All vertices have been successfully deleted.")
                break
        except Exception as e:
            logger.error(f"Failed to delete a batch of vertices: {e}")
            break


async def add_data(
    slice_size=10000, max_workers_vertices=3, max_workers_edges=10, batch_size=100
):
    """
    Add data to the JanusGraph database by inserting vertices and edges.

    Args:
        slice_size (int): Number of entities to process per data slice.
        max_workers_vertices (int): Number of concurrent tasks for vertex upsertion.
        max_workers_edges (int): Number of concurrent tasks for edge upsertion.
        batch_size (int): Number of relationships to process per edge batch.
    """
    data_stream = get_data_stream(entity_subclass_of, slice_size)
    slice_number = 0
    for unique_labels, relationships in data_stream:
        slice_number += 1
        logger.info(
            f"Processing slice {slice_number} with {len(unique_labels)} unique labels and {len(relationships)} relationships."
        )

        # Process all vertices in the current slice
        await process_vertices(unique_labels, max_workers=max_workers_vertices)

        # Process all edges in the current slice
        await process_edges(
            relationships, batch_size=batch_size, max_workers=max_workers_edges
        )


async def delete_data(batch_size=100, method="batch"):
    """
    Delete all data from the JanusGraph database.

    Args:
        batch_size (int): Number of vertices to delete per batch (used in batch deletion).
        method (str): Deletion method - 'all' or 'batch'.
                      'all' deletes all vertices in one operation.
                      'batch' deletes vertices in smaller batches.
    """
    if method == "all":
        # Option 1: Delete all at once
        await delete_all_vertices()
    elif method == "batch":
        # Option 2: Delete in batches to minimize lock contention
        await delete_vertices_in_batches(batch_size=batch_size)
    else:
        logger.error("Invalid deletion method specified. Choose 'all' or 'batch'.")


async def close_client_async():
    """
    Close the Gremlin client synchronously.
    """
    try:
        client.close()
        logger.info("Disconnected from the Gremlin server.")
    except Exception as e:
        logger.error(f"Error while closing client: {e}")


async def main_async(
    operation="add",
    slice_size=10000,
    max_workers_vertices=3,
    max_workers_edges=10,
    batch_size=100,
    delete_method="batch",
):
    """
    Main asynchronous function to perform add or delete operations based on arguments.

    Args:
        operation (str): 'add' to add data or 'delete' to delete all data.
        slice_size (int): Number of entities to process when adding data.
        max_workers_vertices (int): Number of concurrent tasks for vertex upsertion.
        max_workers_edges (int): Number of concurrent tasks for edge upsertion.
        batch_size (int): Number of relationships per edge batch or number of vertices per deletion batch.
        delete_method (str): Method of deletion - 'all' or 'batch'.
    """
    if operation == "add":
        logger.info("Starting data addition process...")
        await add_data(
            slice_size=slice_size,
            max_workers_vertices=max_workers_vertices,
            max_workers_edges=max_workers_edges,
            batch_size=batch_size,
        )
    elif operation == "delete":
        logger.info("Starting data deletion process...")
        await delete_data(batch_size=batch_size, method=delete_method)
    else:
        logger.error("Invalid operation specified. Choose 'add' or 'delete'.")

    # Close the Gremlin connection
    await close_client_async()


def run_operation(
    operation="add",
    slice_size=10000,
    max_workers_vertices=3,
    max_workers_edges=10,
    batch_size=100,
    delete_method="batch",
):
    """
    Run the specified operation asynchronously within the Jupyter notebook.

    Args:
        operation (str): 'add' to add data or 'delete' to delete all data.
        slice_size (int): Number of entities to process when adding data.
        max_workers_vertices (int): Number of concurrent tasks for vertex upsertion.
        max_workers_edges (int): Number of concurrent tasks for edge upsertion.
        batch_size (int): Number of relationships per edge batch or number of vertices per deletion batch.
        delete_method (str): Method of deletion - 'all' or 'batch'.
    """
    try:
        asyncio.run(
            main_async(
                operation,
                slice_size,
                max_workers_vertices,
                max_workers_edges,
                batch_size,
                delete_method,
            )
        )
    except RuntimeError as e:
        logger.error(f"Runtime error: {e}")
    except Exception as e:
        logger.error(f"An error occurred: {e}")


# Example Usage:

# To add data to the JanusGraph database
# Adjust the parameters as needed and uncomment the line below to execute

run_operation(
    operation="add",
    slice_size=10000,  # Number of entities per slice
    max_workers_vertices=10,  # Number of concurrent tasks for vertex upsertion
    max_workers_edges=10,  # Number of concurrent tasks for edge upsertion
    batch_size=1000,  # Number of relationships per edge batch
)

# To delete all data from the JanusGraph database using batch deletion
# Adjust the batch_size and method as needed and uncomment the line below to execute

# run_operation(
#     operation='delete',
#     batch_size=100,                # Number of vertices to delete per batch
#     delete_method='batch'          # Deletion method: 'all' or 'batch'
# )

# To delete all data from the JanusGraph database in one go
# Uncomment the lines below to execute

# run_operation(
#     operation='delete',
#     batch_size=0,                   # Not used in 'all' method
#     delete_method='all'             # Specify deletion method as 'all'
# )